In [49]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to your data directories
train_dir = r'E:\porj\Dataset\train'
validation_dir = r'E:\porj\Dataset\valid'
test_dir = r'E:\porj\Dataset\test'

# Image Data Generators
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Data loading
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

# CNN Model Architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the Model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=176,  # Adjust based on your dataset size
    epochs=15 ,
    validation_data=validation_generator,
    validation_steps=51  # Adjust based on your dataset size
)

# Evaluate the Model
model.evaluate(test_generator)

# Save the Model
model.save('drunk_or_not_model.h5')


Found 3506 images belonging to 2 classes.
Found 1001 images belonging to 2 classes.
Found 501 images belonging to 2 classes.
Epoch 1/15
176/176 [==============================] - 8s 46ms/step - loss: 0.6000 - accuracy: 0.6925 - val_loss: 0.5256 - val_accuracy: 0.7493
Epoch 2/15
176/176 [==============================] - 8s 45ms/step - loss: 0.4843 - accuracy: 0.7673 - val_loss: 0.4629 - val_accuracy: 0.7802
Epoch 3/15
176/176 [==============================] - 8s 44ms/step - loss: 0.4247 - accuracy: 0.8106 - val_loss: 0.4442 - val_accuracy: 0.7902
Epoch 4/15
176/176 [==============================] - 8s 44ms/step - loss: 0.3581 - accuracy: 0.8388 - val_loss: 0.4227 - val_accuracy: 0.7972
Epoch 5/15
176/176 [==============================] - 8s 43ms/step - loss: 0.2872 - accuracy: 0.8728 - val_loss: 0.4341 - val_accuracy: 0.8102
Epoch 6/15
176/176 [==============================] - 8s 44ms/step - loss: 0.2426 - accuracy: 0.8967 - val_loss: 0.3911 - val_accuracy: 0.8402
Epoch 7/15
176/17

In [45]:
!pip3 freeze 

absl-py==2.0.0
anyio==4.2.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
astunparse==1.6.3
async-lru==2.0.4
attrs==23.1.0
Babel==2.14.0
beautifulsoup4==4.12.2
bleach==6.1.0
blinker==1.7.0
cachetools==5.3.2
certifi==2023.11.17
cffi==1.16.0
charset-normalizer==3.3.2
click==8.1.7
colorama==0.4.6
comm==0.2.0
contourpy==1.2.0
cycler==0.12.1
debugpy==1.8.0
decorator==5.1.1
defusedxml==0.7.1
exceptiongroup==1.2.0
executing==2.0.1
fastjsonschema==2.19.0
Flask==3.0.1
flatbuffers==23.5.26
fonttools==4.47.2
fqdn==1.5.1
gast==0.4.0
google-auth==2.25.2
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.60.0
h5py==3.10.0
idna==3.6
importlib-metadata==7.0.1
importlib-resources==6.1.1
ipykernel==6.27.1
ipython==8.18.1
isoduration==20.11.0
itsdangerous==2.1.2
jedi==0.19.1
Jinja2==3.1.2
json5==0.9.14
jsonpointer==2.4
jsonschema==4.20.0
jsonschema-specifications==2023.11.2
jupyter-events==0.9.0
jupyter-lsp==2.2.1
jupyter_client==8.6.0
jupyter_core==5.5.1
jupyter_s

In [43]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


26/26 [==============================] - 1s 38ms/step - loss: 0.5441 - accuracy: 0.8543
Test Accuracy: 85.43%


In [51]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from PIL import Image
from io import BytesIO
import numpy as np

app = Flask(__name__)
model = load_model('drunk_or_not_model.h5')


@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400

    file = request.files['image']
    img_bytes = BytesIO(file.read())
    img = Image.open(img_bytes)
    img = img.resize((150, 150))  # Resize to the size your model expects
    img = img.convert('RGB')      # If your model expects color images

    img_array = np.array(img) / 255.0  # Normalize the image
    img_array = img_array.reshape((1, 150, 150, 3))  # Reshape for the model

    prediction = model.predict(img_array)
    result = 'sober' if prediction[0][0] > 0.5 else 'drunk'

    return jsonify({'prediction': result})



if __name__ == '__main__':
   app.run( debug=True,use_reloader=False)




 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


1/1 [==============================] - 0s 37ms/step


127.0.0.1 - - [31/Jan/2024 16:55:49] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 13ms/step


127.0.0.1 - - [31/Jan/2024 16:55:52] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 12ms/step


127.0.0.1 - - [31/Jan/2024 16:55:55] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 13ms/step


127.0.0.1 - - [31/Jan/2024 16:55:59] "POST /predict HTTP/1.1" 200 -
